In [ ]:
import pandas as pd

In [ ]:
# create giant data object, to house all data
data = {}

In [ ]:
# read all the labels for the data into a df
def read_labels_for_all_houses():
    houses = []
    for house in range(1, 7):
        hi = 'Data/low_freq/house_{}/labels.dat'.format(house)
        df = pd.read_csv(hi, sep=" ", header=None, names = ["appliance_id","appliance_name"], dtype={"appliance_id":"int64","appliance_name":"string"})
        df['house_id'] = house
        houses.append(df)
    return pd.concat(houses).reset_index(drop=True)

labels = read_labels_for_all_houses()
for house in range(1,7):
    print('House {}: \n'.format(house), labels[labels['house_id'] == house] , '\n')

data['labels_for_houses'] = labels

In [ ]:
# read all the appliance data points for all houses
def read_channel_data(labels_for_houses, house_id):
    house_labels = labels_for_houses[labels_for_houses['house_id'] == house_id]
    # print(house_labels)
    df = pd.DataFrame()
    for index,row in house_labels.reset_index().iterrows():
        path = 'data/low_freq/house_{}/'.format(house_id)
        file = path + 'channel_{}.dat'.format(row['appliance_id'])
        file_content = pd.read_table(file, sep = ' ', names = ['unix_time', (row['appliance_id'], row['appliance_name'])], 
                                        dtype = {'unix_time': 'int64', row['appliance_name']:'float64'})
        if(index==0):
            df = file_content
        else:
            # print(index)
            df = pd.merge(df, file_content, how='inner', on='unix_time')
    
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    df = df.set_index(df['timestamp'].values)
    df.drop(['unix_time','timestamp'], axis=1, inplace=True)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=["appliance_id", "appliance_name"])
    return df

# print(data['labels_for_houses'])
data['channels'] = {}
for house in range(1,7):
    print('reading channels for house {}'.format(house))
    data['channels'][house] = read_channel_data(data['labels_for_houses'], house)

print(data)

In [ ]:
# example of indexing with multiindex columns

# first retrieve house df (house 2)
house_channels = data['channels'][2]

# print columns
print(house_channels.columns)

# print only names
print(house_channels.columns.get_level_values('appliance_name'))

# print only ids
print(house_channels.columns.get_level_values('appliance_id'))

# index using appliance name 
print(house_channels.xs('mains',level='appliance_name', axis=1).head())

# and using appliance id
print(house_channels.xs(2,level='appliance_id', axis=1).head())
